# Binary Answer Type

This notebook demonstrates how to generate forecasting questions with binary (yes/no) answers. Binary questions are the simplest and most common type, perfect for event prediction, outcome forecasting, and decision-making scenarios.

In [ ]:
%pip install lightningrod-ai

from IPython.display import clear_output
clear_output()

In [ ]:
import os
from lightningrod import LightningRod

api_key = os.getenv("LIGHTNINGROD_API_KEY")
if not api_key:
    raise ValueError("LIGHTNINGROD_API_KEY is not set")

client = LightningRod(api_key=api_key)

## Binary Answer Type Overview

Binary questions have two possible answers: Yes/No or True/False. They're ideal for:
- Event prediction ("Will X happen?")
- Outcome forecasting ("Will Y succeed?")
- Decision-making scenarios ("Should we do Z?")

Binary questions work with any data source: Google News, GDELT, or custom documents.

In [ ]:
from datetime import datetime, timedelta
from lightningrod import AnswerType, AnswerTypeEnum, NewsSeedGenerator

answer_type = AnswerType(answer_type=AnswerTypeEnum.BINARY)

seed_generator = NewsSeedGenerator(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now(),
    interval_duration_days=7,
    search_query="technology product launches",
)

In [ ]:
from lightningrod import QuestionGenerator, FilterCriteria

question_generator = QuestionGenerator(
    instructions=(
        "Generate binary forecasting questions about technology product launches. "
        "Questions should be answerable with Yes or No."
    ),
    examples=[
        "Will Apple release a new iPhone this year?",
        "Will Google launch a new AI product in Q2?",
        "Will Microsoft release Windows 12 in 2025?",
    ],
    bad_examples=[
        "What features will the new iPhone have?",
        "When will Google launch the product?",
        "How much will Windows 12 cost?",
    ],
    filter_=FilterCriteria(
        rubric="The question must be answerable with Yes or No",
        min_score=0.7
    ),
    answer_type=answer_type,
)

In [ ]:
from lightningrod import WebSearchLabeler, QuestionRenderer

labeler = WebSearchLabeler(
    answer_type=answer_type,
    confidence_threshold=0.5,
)

renderer = QuestionRenderer(answer_type=answer_type)

## Run the Pipeline

Generate binary questions from the news articles.

In [ ]:
from lightningrod import QuestionPipeline

pipeline_config = QuestionPipeline(
    seed_generator=seed_generator,
    question_generator=question_generator,
    labeler=labeler,
    renderer=renderer,
)

dataset = client.transforms.run(pipeline_config, max_questions=30)

## View Results

Inspect the generated questions and answers. Each sample contains `seed`, `question`, `label`, `prompt`, and optional `context` and `meta` fields. See [API.md](../API.md) for the complete sample structure.

In [ ]:
%pip install pandas

from IPython.display import clear_output
clear_output()

In [ ]:
import pandas as pd

# Download samples to memory
samples = dataset.download()
print(f"Generated {dataset.num_rows} samples\n")

# Convert cached samples to a list of dictionaries
rows = dataset.flattened()

df = pd.DataFrame(rows)
print(df.head())

## When to Use Binary vs Other Answer Types

**Use Binary when:**
- You need simple yes/no answers
- Predicting events or outcomes
- Building decision-making systems
- You want the simplest evaluation

**Consider other types when:**
- You need numeric values → Use Continuous
- You need multiple options → Use Multiple Choice
- You need open-ended explanations → Use Free Response